Make the import

In [1]:
import os
import logging
import numpy as np
import mne
import myonset as myo

Set the path

In [2]:
#path_bdf = os.path.join('.','bdf')
#path_mrk = os.path.join('.','automatic_detection')
path_bdf = os.path.join('.')
path_mrk = os.path.join('.')

Set trigger id values: to adapt to your experiment

In [3]:
#triggers values used for trial segmentation (e.g., stimulus or fixation cross)
stim_id = {'red_left':    12,\
           'red_right':   22,\
           'green_left':  11,\
           'green_right': 21}

#triggers values used for detected onset, offset and peak events
emg_id = {'onset': 131, 'offset': 132, 'peak': 133}

# list of response triggers, used to remove bursts detected after response 
resp_list =  [100,200] 

# Load the file

Set EMG file name

In [4]:
name_bdf  = 's1.bdf'

Extract file name and create log file

In [5]:
nameSubj = name_bdf.split('.')[0]
fname = os.path.join(path_bdf,name_bdf)

logName = os.path.join(path_bdf,nameSubj + '.log')
logging.basicConfig(filename=logName, level=logging.INFO) # needed in jupyter notebook
mne.set_log_file(logName,overwrite=True)
logging.info("Automatic onsets/offsets detection:")

Load raw data

In [6]:
raw = mne.io.read_raw_bdf(fname, preload=True,stim_channel = 'Status')

In [7]:
raw

<RawEDF | s1.bdf, 5 x 118784 (58.0 s), ~4.5 MB, data loaded>

Extract events 

In [8]:
mne_events = mne.find_events(raw, shortest_event=1)

Needed when default Status value is not zero.

In [9]:
#mne_events[:,2] = mne_events[:,2]-mne_events[:,1] 
mne_events[:,2] = mne_events[:,2]-mne_events[1,1] 


# Pre-process signal

In [10]:
raw.info['ch_names']

['EXG1', 'EXG2', 'EXG3', 'EXG4', 'Status']

EMG bipolar reference 

In [11]:
mne.set_bipolar_reference(raw,anode=['EXG1','EXG3'], cathode=['EXG2','EXG4'],ch_name=['EMG_L','EMG_R'],\
copy=False)

<RawEDF | s1.bdf, 3 x 118784 (58.0 s), ~2.7 MB, data loaded>

Set channels

In [12]:
# if non ergos
raw.pick(['EMG_L','EMG_R'])
raw.set_channel_types({'EMG_L':'emg',
                       'EMG_R':'emg'
                      })

emg_channels_idx = {0: 'EMG_L', 1: 'EMG_R'}
channel_names = list(emg_channels_idx.values()) 

Filter EMG signals

In [13]:
raw = myo.use_mne.apply_filter(raw, ch_names = ['EMG_L','EMG_R'],low_cutoff = 10)
logging.info("\tHigh pass filtering of EMG traces at 10Hz")

Get pre-processed raw data

In [14]:
data_raw = raw.get_data(channel_names)

# Segment based on events

Put mne events in our events structure

In [15]:
events = myo.Events(sample=mne_events[:,0], code=mne_events[:,2], chan=[-1]*mne_events.shape[0],\
                    sf=raw.info['sfreq'])

Define segmentation window

In [16]:
tmin = -.5
tmax = 1.5
epoch_time = myo.times(tmin,tmax,events.sf)
t0 = mne.find_times(0,epoch_time)
#tmax_sample = dbt.find_times(tmax,epoch_time)

AttributeError: module 'mne' has no attribute 'times'

Segment and extract data epochs

In [ ]:
events.sf

In [ ]:
epochs_events = events.segment(code_t0=list(stim_id.values()), tmin=tmin, tmax=tmax)
data_epochs = epochs_events.get_data(data_raw)

# Run automatic detection

Set threshold used for raw and Teager-Kaiser EMG var_onset detection

In [ ]:
thEMG_raw = 8 
thEMG_tk = 10 

logging.info("\t\t threshold for EMG left: " + str(thEMG_raw))
logging.info("\t\t threshold for EMG right: " + str(thEMG_raw))        

logging.info("\t\t threshold for EMG left Teager-Kaiser: " + str(thEMG_tk))
logging.info("\t\t threshold for EMG right Teager-Kaiser: " + str(thEMG_tk))


Compute global variance, this will not be used most of the time

In [ ]:
mBlRaw,stBlRaw = myo.global_var(data_epochs,epoch_time,cor_var = 2.5,use_tkeo = False)
mBlTk,stBlTk = myo.global_var(data_epochs,epoch_time,cor_var = 2.5,use_tkeo = True)

Big loop doing the detection

In [ ]:
for e in range(epochs_events.nb_trials()):
    # Onset and offset EMG detection
    for c in emg_channels_idx.keys():
        
        current = data_epochs[e,c,:]
        
        #Lcal mBl and stBl are recommended, to use global values, use mBlRaw/mBlTk[c] and stBlRaw/stBlTk[c] computed above
        onsets,offsets = myo.get_onsets(current, epoch_time, sf=events.sf,\
                                        th_raw= thEMG_raw, use_raw=True, time_limit_raw=.025, min_samples_raw=5,\
                                        varying_min_raw=1, mbsl_raw=None, stbsl_raw=None, \
                                        th_tkeo= thEMG_tk, use_tkeo=True, time_limit_tkeo=.025,  min_samples_tkeo=5,\
                                        varying_min_tkeo=0, mbsl_tkeo=None, stbsl_tkeo=None)
        
        # Remove burst starting and ending before time 0
        onsets = [onsets[b] for b in range(len(onsets)) if (offsets[b] > t0)]
        offsets = [offsets[b] for b in range(len(offsets)) if (offsets[b] > t0)]
        # If one onset remains before t0, put its latency to time 0
        onsets = [np.max((b,t0+1)) for b in onsets]
        
        # Remove bursts starting after the first response
        stim = epochs_events.list_evts_trials[e].find_events(code=list(stim_id.values()))
        resp = epochs_events.list_evts_trials[e].find_events(code=resp_list)
        if len(resp) > 0:
            #latency of the first response after the first stimulus
            resp_latency =  epochs_events.list_evts_trials[e].lat.sample[resp[resp > stim[0]][0]]
        else: 
            resp_latency = tmax_sample # if no response, resp latency is equal to tmax
        offsets = [offsets[b] for b in range(len(offsets)) if (onsets[b] < resp_latency)]
        onsets = [onsets[b] for b in range(len(onsets)) if (onsets[b] < resp_latency)]

        # Put in event structure
        onsets_events = myo.Events(sample=onsets, time=epoch_time[onsets],\
                                   code=[emg_id['onset']]*len(onsets), chan=[c]*len(onsets), sf=epochs_events.sf) 
        offsets_events = myo.Events(sample=offsets, time=epoch_time[offsets],\
                                    code=[emg_id['offset']]*len(offsets), chan=[c]*len(offsets), sf=epochs_events.sf) 
        
        # Add in epochs events
        epochs_events.list_evts_trials[e].add_events(onsets_events)
        epochs_events.list_evts_trials[e].add_events(offsets_events)
        

# Save in new marker file

First put epoch events in continuous time

In [ ]:
continuous_events_with_detection = epochs_events.as_continuous(drop_duplic=True)[0]

Add those to the original events

In [ ]:
events.add_events(continuous_events_with_detection, drop_duplic=True)

Save in folder path_mrk (defined above)

In [ ]:
fname_detected_mrk = os.path.join(path_mrk, nameSubj+'_detect_emg.csv')
events.to_csv(fname_detected_mrk,\
              header='sample,time,code,chan', sep=',',\
              save_sample=True, save_time=True, save_code=True, save_chan=True)

logging.info("\tEvents saved in " + fname_detected_mrk) 
